## Lojistik Regresyon Modeli Oluşturma - Ders 4

![Lojistik ve doğrusal regresyon infografiği](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Ders Öncesi Test](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Giriş

Regresyon üzerine olan bu son derste, temel *klasik* ML tekniklerinden biri olan Lojistik Regresyonu inceleyeceğiz. Bu tekniği, ikili kategorileri tahmin etmek için desenleri keşfetmek amacıyla kullanabilirsiniz. Bu şeker çikolata mı, değil mi? Bu hastalık bulaşıcı mı, değil mi? Bu müşteri bu ürünü seçecek mi, seçmeyecek mi?

Bu derste şunları öğreneceksiniz:

-   Lojistik regresyon teknikleri

✅ Bu tür regresyonla çalışmayı daha iyi anlamak için şu [Learn modülüne](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott) göz atabilirsiniz.

## Ön Koşul

Balkabağı verileriyle çalıştıktan sonra, üzerinde çalışabileceğimiz bir ikili kategori olduğunu fark edecek kadar aşina olduk: `Renk`.

Şimdi, bazı değişkenlere dayanarak *belirli bir balkabağının muhtemelen hangi renkte olduğunu* (turuncu 🎃 veya beyaz 👻) tahmin etmek için bir lojistik regresyon modeli oluşturalım.

> Neden regresyonla ilgili bir ders grubunda ikili sınıflandırmadan bahsediyoruz? Sadece dilsel kolaylık açısından, çünkü lojistik regresyon [aslında bir sınıflandırma yöntemi](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), ancak doğrusal tabanlı bir yöntemdir. Verileri sınıflandırmanın diğer yollarını bir sonraki ders grubunda öğrenin.

Bu ders için aşağıdaki paketlere ihtiyacımız olacak:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/), veri bilimini daha hızlı, kolay ve eğlenceli hale getirmek için tasarlanmış bir [R paketleri koleksiyonudur](https://www.tidyverse.org/packages).

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) çerçevesi, modelleme ve makine öğrenimi için bir [paketler koleksiyonudur](https://www.tidymodels.org/packages/).

-   `janitor`: [janitor paketi](https://github.com/sfirke/janitor), kirli verileri incelemek ve temizlemek için basit araçlar sağlar.

-   `ggbeeswarm`: [ggbeeswarm paketi](https://github.com/eclarke/ggbeeswarm), ggplot2 kullanarak beeswarm tarzı grafikler oluşturmak için yöntemler sunar.

Bu paketleri şu şekilde yükleyebilirsiniz:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternatif olarak, aşağıdaki script, bu modülü tamamlamak için gerekli paketlere sahip olup olmadığınızı kontrol eder ve eksikse sizin için yükler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Soruyu Tanımlayın**

Bizim amacımız için bunu bir ikili olarak ifade edeceğiz: 'Beyaz' veya 'Beyaz Değil'. Veri setimizde ayrıca 'çizgili' bir kategori var, ancak çok az örneği olduğu için bunu kullanmayacağız. Zaten veri setinden eksik değerleri çıkardığımızda bu kategori kayboluyor.

> 🎃 Eğlenceli bilgi, bazen beyaz balkabaklarına 'hayalet' balkabakları diyoruz. Oyması çok kolay değil, bu yüzden turuncu olanlar kadar popüler değiller ama oldukça havalı görünüyorlar! Bu yüzden sorumuzu şu şekilde de yeniden formüle edebiliriz: 'Hayalet' veya 'Hayalet Değil'. 👻

## **Lojistik Regresyon Hakkında**

Lojistik regresyon, daha önce öğrendiğiniz doğrusal regresyondan birkaç önemli şekilde farklıdır.

#### **İkili Sınıflandırma**

Lojistik regresyon, doğrusal regresyonla aynı özellikleri sunmaz. İlki, `ikili bir kategori` ("turuncu veya turuncu değil") hakkında bir tahmin sunarken, ikincisi `sürekli değerler` tahmin edebilir, örneğin bir balkabağının kökeni ve hasat zamanı verildiğinde, *fiyatının ne kadar artacağı*.

![Dasani Madipalli tarafından hazırlanan infografik](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Diğer Sınıflandırmalar

Lojistik regresyonun başka türleri de vardır, bunlar arasında çok kategorili ve sıralı olanlar bulunur:

- **Çok kategorili**, birden fazla kategoriye sahip olmayı içerir - "Turuncu, Beyaz ve Çizgili".

- **Sıralı**, mantıksal olarak sıralanmış kategorileri içerir, örneğin sonuçlarımızı sınırlı sayıda boyuta göre sıralamak istersek (mini, küçük, orta, büyük, xl, xxl).

![Çok kategorili vs sıralı regresyon](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Değişkenlerin İLİŞKİLİ Olması GEREKMEZ**

Doğrusal regresyonun daha fazla ilişkili değişkenlerle daha iyi çalıştığını hatırlıyor musunuz? Lojistik regresyon bunun tersidir - değişkenlerin uyumlu olması gerekmez. Bu, zayıf korelasyonlara sahip olan bu veri için işe yarar.

#### **Çok Temiz Veriye İhtiyacınız Var**

Lojistik regresyon, daha fazla veri kullanırsanız daha doğru sonuçlar verir; küçük veri setimiz bu görev için ideal değil, bunu aklınızda bulundurun.

✅ Lojistik regresyona uygun olabilecek veri türlerini düşünün

## Alıştırma - Veriyi Düzenleyin

Öncelikle, eksik değerleri çıkararak ve yalnızca bazı sütunları seçerek veriyi biraz temizleyin:

1. Aşağıdaki kodu ekleyin:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Yeni veri çerçevenize her zaman bir göz atabilirsiniz, aşağıdaki gibi [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) fonksiyonunu kullanarak:


In [ ]:
pumpkins_select %>% 
  glimpse()


İkili sınıflandırma problemi yapacağımızı doğrulayalım:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Görselleştirme - kategorik grafik
Şimdiye kadar balkabağı verilerini tekrar yüklediniz ve birkaç değişkeni içeren bir veri setini koruyacak şekilde temizlediniz, bunlar arasında Renk de bulunuyor. Hadi ggplot kütüphanesini kullanarak veri çerçevesini not defterinde görselleştirelim.

ggplot kütüphanesi, verilerinizi görselleştirmek için bazı güzel yöntemler sunar. Örneğin, her Çeşit ve Renk için verilerin dağılımlarını kategorik bir grafikte karşılaştırabilirsiniz.

1. Balkabağı verilerimizi kullanarak, her balkabağı kategorisi (turuncu veya beyaz) için bir renk eşlemesi belirterek geombar fonksiyonunu kullanarak böyle bir grafik oluşturun:


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Verilere bakarak, Renk verilerinin Çeşitlilik ile nasıl ilişkili olduğunu görebilirsiniz.

✅ Bu kategorik grafiğe bakarak, hangi ilginç keşifleri hayal edebilirsiniz?


### Veri Ön İşleme: Özellik Kodlama

Kabak veri setimizdeki tüm sütunlar metin değerleri içeriyor. Kategorik verilerle çalışmak insanlar için sezgisel olsa da makineler için aynı durum geçerli değil. Makine öğrenimi algoritmaları sayılarla daha iyi çalışır. Bu nedenle, kodlama veri ön işleme aşamasında çok önemli bir adımdır; çünkü kategorik verileri sayısal verilere dönüştürmemizi sağlar ve bu süreçte hiçbir bilgi kaybı yaşanmaz. İyi bir kodlama, iyi bir model oluşturmanın temelini oluşturur.

Özellik kodlama için iki ana kodlayıcı türü vardır:

1. **Ordinal kodlayıcı**: Bu kodlayıcı, sıralı değişkenler için uygundur. Sıralı değişkenler, verilerin mantıksal bir sıralamayı takip ettiği kategorik değişkenlerdir. Örneğin, veri setimizdeki `item_size` sütunu gibi. Bu kodlayıcı, her kategorinin bir sayı ile temsil edildiği bir eşleme oluşturur. Bu sayı, sütundaki kategorinin sırasını ifade eder.

2. **Kategorik kodlayıcı**: Bu kodlayıcı, nominal değişkenler için uygundur. Nominal değişkenler, verilerin mantıksal bir sıralamayı takip etmediği kategorik değişkenlerdir. Örneğin, veri setimizdeki `item_size` dışındaki tüm özellikler. Bu kodlama yöntemi "one-hot encoding" olarak adlandırılır. Her kategori, ikili bir sütunla temsil edilir: kodlanmış değişken, kabak o çeşide aitse 1, değilse 0 değerini alır.

Tidymodels, veri ön işleme için başka bir kullanışlı paket sunar: [recipes](https://recipes.tidymodels.org/) - veri ön işleme için bir paket. Bir `recipe` tanımlayacağız; bu, tüm tahmin edici sütunların bir dizi tam sayıya kodlanması gerektiğini belirtir. Ardından, gerekli miktarları ve istatistikleri tahmin etmek için `prep` işlemini gerçekleştireceğiz ve son olarak yeni verilere hesaplamaları uygulamak için `bake` işlemini kullanacağız.

> Normalde, recipes genellikle modelleme için bir ön işleyici olarak kullanılır. Bu durumda, bir veri setine modelleme için hazır hale getirilmesi amacıyla hangi adımların uygulanması gerektiğini tanımlar. Bu durumda, bir tarifi manuel olarak `prep` ve `bake` ile tahmin etmek yerine bir `workflow()` kullanmanız **şiddetle tavsiye edilir**. Bunu birazdan daha ayrıntılı göreceğiz.
>
> Ancak şu an için, recipes + prep + bake kullanarak bir veri setine hangi adımların uygulanması gerektiğini belirliyoruz. Bu adımları uygulayarak ön işlenmiş veriyi çıkarıyoruz.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Öğe Boyutu sütunu için bir sıralı kodlayıcı kullanmanın avantajları nelerdir?

### Değişkenler arasındaki ilişkileri analiz etme

Artık verilerimizi ön işleme tabi tuttuğumuza göre, özellikler ve etiket arasındaki ilişkileri analiz ederek modelin, özellikler verildiğinde etiketi ne kadar iyi tahmin edebileceği hakkında bir fikir edinebiliriz. Bu tür bir analizi gerçekleştirmenin en iyi yolu verileri görselleştirmektir. 
Bu analiz için tekrar ggplot geom_boxplot_ fonksiyonunu kullanacağız ve Öğe Boyutu, Çeşit ve Renk arasındaki ilişkileri kategorik bir grafikte görselleştireceğiz. Verileri daha iyi görselleştirmek için kodlanmış Öğe Boyutu sütununu ve kodlanmamış Çeşit sütununu kullanacağız.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Bir swarm plot kullanın

Renk, ikili bir kategori olduğu için (Beyaz veya Değil), görselleştirme için '[özel bir yaklaşım](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' gerektirir.

Renk dağılımını item_size ile ilişkili olarak göstermek için bir `swarm plot` deneyin.

[ggbeeswarm paketi](https://github.com/eclarke/ggbeeswarm)'ni kullanacağız. Bu paket, ggplot2 ile arı kovanı tarzı grafikler oluşturmak için yöntemler sağlar. Arı kovanı grafikleri, normalde üst üste binecek noktaları yan yana düşecek şekilde düzenlemenin bir yoludur.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Artık renklerin ikili kategorileri ile daha büyük boyut grubu arasındaki ilişki hakkında bir fikrimiz olduğuna göre, bir kabağın muhtemel rengini belirlemek için lojistik regresyonu inceleyelim.

## Modelinizi oluşturun

Sınıflandırma modelinizde kullanmak istediğiniz değişkenleri seçin ve veriyi eğitim ve test setlerine ayırın. [rsample](https://rsample.tidymodels.org/), Tidymodels içinde yer alan bir paket, verilerin verimli bir şekilde bölünmesi ve yeniden örneklenmesi için altyapı sağlar:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Artık modeli, eğitim özelliklerini eğitim etiketi (renk) ile eşleştirerek eğitmeye hazırız.

Verilerimizi modellemeye hazırlamak için yapılması gereken ön işleme adımlarını belirten bir tarif oluşturarak başlayacağız, yani: kategorik değişkenleri bir dizi tam sayıya kodlama. Tıpkı `baked_pumpkins` gibi, bir `pumpkins_recipe` oluşturuyoruz ancak bunu `prep` ve `bake` yapmıyoruz çünkü bu, birkaç adım sonra göreceğiniz bir iş akışına dahil edilecek.

Tidymodels'de lojistik regresyon modelini belirtmenin oldukça fazla yolu vardır. `?logistic_reg()`'e bakabilirsiniz. Şimdilik, varsayılan `stats::glm()` motoru aracılığıyla bir lojistik regresyon modeli belirteceğiz.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Artık bir tarif ve model spesifikasyonuna sahip olduğumuza göre, bunları bir araya getirerek bir nesne oluşturmanın bir yolunu bulmamız gerekiyor. Bu nesne, önce veriyi ön işleme tabi tutacak (arka planda hazırlık + pişirme), modeli ön işlenmiş veri üzerinde eğitecek ve ayrıca olası son işlem aktivitelerine izin verecek.

Tidymodels'da, bu kullanışlı nesne [`workflow`](https://workflows.tidymodels.org/) olarak adlandırılır ve modelleme bileşenlerinizi pratik bir şekilde bir arada tutar.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Bir iş akışı *belirlendikten* sonra, bir model [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) fonksiyonu kullanılarak `eğitilebilir`. İş akışı, bir tarifi tahmin edecek ve verileri eğitmeden önce ön işleyecektir, bu yüzden bunu manuel olarak prep ve bake kullanarak yapmamıza gerek kalmayacak.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Modelin eğitim sırasında öğrendiği katsayılar çıktı olarak gösterilir.

Artık modeli eğitim verileriyle eğittik, test verileri üzerinde tahminler yapabiliriz. Bunun için [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) fonksiyonunu kullanabiliriz. Hadi başlayalım ve modelimizi test seti için etiketleri ve her etiket için olasılıkları tahmin etmek için kullanalım. Olasılık 0.5'ten büyük olduğunda tahmin edilen sınıf `WHITE`, aksi takdirde `ORANGE` olur.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Çok güzel! Bu, lojistik regresyonun nasıl çalıştığına dair daha fazla bilgi sağlıyor.

### Bir karmaşıklık matrisi ile daha iyi anlama

Her tahmini, karşılık gelen "gerçek değer" ile karşılaştırmak, modelin ne kadar iyi tahmin yaptığını belirlemek için çok verimli bir yöntem değildir. Neyse ki, Tidymodels'in elinde birkaç numara daha var: [`yardstick`](https://yardstick.tidymodels.org/) - performans metriklerini kullanarak modellerin etkinliğini ölçmek için kullanılan bir paket.

Sınıflandırma problemleriyle ilişkili performans metriklerinden biri [`karmaşıklık matrisi`](https://wikipedia.org/wiki/Confusion_matrix). Bir karmaşıklık matrisi, bir sınıflandırma modelinin ne kadar iyi performans gösterdiğini açıklar. Karmaşıklık matrisi, her sınıftaki kaç örneğin model tarafından doğru bir şekilde sınıflandırıldığını tablo halinde gösterir. Bizim durumumuzda, kaç turuncu balkabağının turuncu olarak sınıflandırıldığını ve kaç beyaz balkabağının beyaz olarak sınıflandırıldığını gösterecek; ayrıca karmaşıklık matrisi, **yanlış** kategorilere sınıflandırılanların sayısını da gösterir.

Yardstick paketindeki [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) fonksiyonu, gözlemlenen ve tahmin edilen sınıfların bu çapraz tablosunu hesaplar.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Hadi karışıklık matrisini yorumlayalım. Modelimizden balkabaklarını iki ikili kategori arasında sınıflandırması isteniyor: kategori `beyaz` ve kategori `beyaz değil`.

-   Eğer modeliniz bir balkabağını beyaz olarak tahmin ederse ve gerçekte 'beyaz' kategorisine aitse, buna `doğru pozitif` denir ve bu, sol üstteki sayı ile gösterilir.

-   Eğer modeliniz bir balkabağını beyaz değil olarak tahmin ederse ve gerçekte 'beyaz' kategorisine aitse, buna `yanlış negatif` denir ve bu, sol alttaki sayı ile gösterilir.

-   Eğer modeliniz bir balkabağını beyaz olarak tahmin ederse ve gerçekte 'beyaz değil' kategorisine aitse, buna `yanlış pozitif` denir ve bu, sağ üstteki sayı ile gösterilir.

-   Eğer modeliniz bir balkabağını beyaz değil olarak tahmin ederse ve gerçekte 'beyaz değil' kategorisine aitse, buna `doğru negatif` denir ve bu, sağ alttaki sayı ile gösterilir.

| Gerçek |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Tahmin**    | BEYAZ  | TURUNCU |
| BEYAZ         | DP     | YP      |
| TURUNCU       | YN     | DN      |

Tahmin edebileceğiniz gibi, daha fazla doğru pozitif ve doğru negatif sayısına sahip olmak ve daha az yanlış pozitif ve yanlış negatif sayısına sahip olmak tercih edilir, bu da modelin daha iyi performans gösterdiğini ifade eder.

Karışıklık matrisi faydalıdır çünkü bir sınıflandırma modelinin performansını daha iyi değerlendirmemize yardımcı olabilecek diğer metriklerin ortaya çıkmasını sağlar. Şimdi bunlardan bazılarını inceleyelim:

🎓 Kesinlik: `DP/(DP + YP)` tahmin edilen pozitiflerin gerçekten pozitif olma oranı olarak tanımlanır. Ayrıca [pozitif öngörü değeri](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value") olarak da adlandırılır.

🎓 Duyarlılık: `DP/(DP + YN)` gerçekte pozitif olan örnekler arasından pozitif sonuçların oranı olarak tanımlanır. Ayrıca `hassasiyet` olarak da bilinir.

🎓 Özgüllük: `DN/(DN + YP)` gerçekte negatif olan örnekler arasından negatif sonuçların oranı olarak tanımlanır.

🎓 Doğruluk: `DP + DN/(DP + DN + YP + YN)` Bir örnek için doğru bir şekilde tahmin edilen etiketlerin yüzdesi.

🎓 F Ölçüsü: Kesinlik ve duyarlılığın ağırlıklı ortalaması, en iyi değer 1 ve en kötü değer 0'dır.

Haydi bu metrikleri hesaplayalım!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Bu modelin ROC eğrisini görselleştirin

Hadi, sözde [`ROC eğrisi`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)'ni görmek için bir görselleştirme daha yapalım:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC eğrileri, bir sınıflandırıcının çıktısını doğru ve yanlış pozitifler açısından değerlendirmek için sıklıkla kullanılır. ROC eğrileri genellikle Y ekseninde `Doğru Pozitif Oranı`/Duyarlılık ve X ekseninde `Yanlış Pozitif Oranı`/1-Özgüllük gösterir. Bu nedenle, eğrinin dikliği ve orta çizgi ile eğri arasındaki boşluk önemlidir: eğrinin hızla yukarı çıkıp çizginin üzerine geçmesini istersiniz. Bizim durumumuzda, başlangıçta yanlış pozitifler vardır ve ardından çizgi düzgün bir şekilde yukarı çıkıp geçer.

Son olarak, gerçek Eğri Altındaki Alanı hesaplamak için `yardstick::roc_auc()` kullanabiliriz. AUC'yi yorumlamanın bir yolu, modelin rastgele bir pozitif örneği rastgele bir negatif örnekten daha yüksek sıralama olasılığı olarak değerlendirilmesidir.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Sonuç yaklaşık `0.975`. AUC'nin 0 ile 1 arasında değiştiğini göz önünde bulundurursak, yüksek bir skor elde etmek istersiniz çünkü tahminlerinde %100 doğru olan bir modelin AUC değeri 1 olur; bu durumda model *oldukça iyi*.

Gelecekteki sınıflandırma derslerinde, modelinizin skorlarını nasıl iyileştirebileceğinizi öğreneceksiniz (bu durumda dengesiz veriyle başa çıkmak gibi).

## 🚀Meydan Okuma

Lojistik regresyon hakkında keşfedilecek çok şey var! Ancak öğrenmenin en iyi yolu denemektir. Bu tür bir analize uygun bir veri seti bulun ve onunla bir model oluşturun. Ne öğreniyorsunuz? ipucu: İlginç veri setleri için [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets)'ı deneyin.

## Gözden Geçirme ve Kendi Kendine Çalışma

Stanford'dan [bu makalenin](https://web.stanford.edu/~jurafsky/slp3/5.pdf) ilk birkaç sayfasını okuyarak lojistik regresyonun bazı pratik kullanım alanlarını öğrenin. Şimdiye kadar incelediğimiz regresyon türlerinden hangisinin hangi görevler için daha uygun olduğunu düşünün. Hangisi daha iyi çalışır?



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
